<a href="https://colab.research.google.com/github/spolivin/arxiv-summarizer/blob/master/lora_finetuning/arxiv_summarization_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing the environment

In [ ]:
!pip install datasets evaluate rouge_score --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
import warnings

import evaluate
import numpy as np
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    logging,
)

CKPT_PATH = "facebook/bart-large-cnn" # Path to the fine-tuned base model
DATASET_PATH = "ccdv/arxiv-summarization" # Path to the dataset to be used for fine-tuning
HF_REPO_PATH = "spolivin/bart-arxiv-lora" # Path to the repo where LoRA adapters are saved

warnings.filterwarnings("ignore")
logging.set_verbosity_error()

# Loading and preprocessing data

In [ ]:
arxiv_train = load_dataset(DATASET_PATH, split="train[:5000]")
arxiv_valid = load_dataset(DATASET_PATH, split="validation[:300]")

README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

train-00000-of-00015.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

train-00001-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00002-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00003-of-00015.parquet:   0%|          | 0.00/227M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [ ]:
arxiv_train

Dataset({
    features: ['article', 'abstract'],
    num_rows: 5000
})

In [ ]:
arxiv_valid

Dataset({
    features: ['article', 'abstract'],
    num_rows: 300
})

In [ ]:
# Loading the tokenizer for the base model
tokenizer = AutoTokenizer.from_pretrained(CKPT_PATH)

def tokenizing_function(examples):
    """Tokenizes article and abstract texts."""
    # Tokenizing article text
    model_inputs = tokenizer(examples["article"], max_length=1024, truncation=True)
    # Tokenizing abstrac text (label)
    labels = tokenizer(text_target=examples["abstract"], max_length=150, truncation=True)
    # Adding label tokens as a label column to model inputs
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenized_arxiv_train = arxiv_train.map(tokenizing_function, batched=True)
tokenized_arxiv_valid = arxiv_valid.map(tokenizing_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

# Setting up model with LoRA

In [ ]:
# Loading base model from HF
model = AutoModelForSeq2SeqLM.from_pretrained(CKPT_PATH)
print(f"Model size ({CKPT_PATH}): {model.get_memory_footprint() / 1e9:.2f} GB")

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Model size (facebook/bart-large-cnn): 1.63 GB


In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 407,470,080 || trainable%: 0.2895


# Preparation for fine-tuning

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    return_tensors="pt",
)

In [ ]:
# Loading ROUGE metric
metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    """Computes ROUGE metrics."""
    # Retrieving predictions and labels
    predictions, labels = eval_pred

    # Decoding the predicted tokens
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Decoding labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Computing ROUGE metrics for the batch
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result = {key: value * 100 for key, value in result.items()}

    # Adding meaned generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Running fine-tuning

In [ ]:
torch.cuda.empty_cache()

training_args = Seq2SeqTrainingArguments(
    output_dir=HF_REPO_PATH.split("/")[-1],
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    warmup_ratio=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    num_train_epochs=6,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge2",
    greater_is_better=True,
    fp16=True,
    push_to_hub=False,
    report_to="none",
    disable_tqdm=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_arxiv_train,
    eval_dataset=tokenized_arxiv_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.701424,40.975100,13.085800,22.721800,34.009600,138.116700
2,No log,2.633963,41.985400,13.917000,23.440300,35.034800,138.423300
3,No log,2.608167,42.380300,14.121800,23.633300,35.320200,136.483300
4,2.857500,2.593262,42.106800,14.137600,23.758100,35.028400,137.106700
5,2.857500,2.588403,42.297900,13.950300,23.662500,35.186700,137.140000


TrainOutput(global_step=936, training_loss=2.759424845377604, metrics={'train_runtime': 5295.7373, 'train_samples_per_second': 5.665, 'train_steps_per_second': 0.177, 'total_flos': 6.486527958633677e+16, 'train_loss': 2.759424845377604, 'epoch': 5.9664})

In [ ]:
model.push_to_hub(HF_REPO_PATH)

adapter_model.safetensors:   0%|          | 0.00/4.74M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/spolivin/bart-arxiv-lora/commit/e9af83dd8a453e5970aa277be89d1c344e0c918c', commit_message='Upload model', commit_description='', oid='e9af83dd8a453e5970aa277be89d1c344e0c918c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spolivin/bart-arxiv-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='spolivin/bart-arxiv-lora'), pr_revision=None, pr_num=None)

# Testing the fine-tuned model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CKPT_PATH)
base_model = AutoModelForSeq2SeqLM.from_pretrained(CKPT_PATH)
lora_model = PeftModel.from_pretrained(base_model, HF_REPO_PATH)

original_model = AutoModelForSeq2SeqLM.from_pretrained(CKPT_PATH)

adapter_config.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.74M [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset(DATASET_PATH, split="test")

In [ ]:
def generate_summary(model, tokenizer, text, max_length=150):
    """Generates summary for the input text."""
    # Tokenizing input text
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=1024,
    ).to("cuda" if torch.cuda.is_available() else "cpu")
    # Moving model to GPU
    model.to(inputs.input_ids.device)
    # Generating output tokens
    summary_ids = model.generate(**inputs, max_length=max_length, num_beams=4, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# Testing on 100 test articles
num_samples = 100
articles = dataset["article"][:num_samples] # test articles
ground_truth_summaries = dataset["abstract"][:num_samples] # test abstracts (labels)

# Generated summaries (original model)
original_summaries = [generate_summary(original_model, tokenizer, text) for text in articles]

# Generated summaries (fine-tuned LoRA)
finetuned_summaries = [generate_summary(lora_model, tokenizer, text) for text in articles]

In [ ]:
# Compute ROUGE scores
rouge_original = metric.compute(predictions=original_summaries, references=ground_truth_summaries, use_stemmer=True)
rouge_finetuned = metric.compute(predictions=finetuned_summaries, references=ground_truth_summaries, use_stemmer=True)

# Print results
print("Original Model ROUGE Scores:")
print(rouge_original)
print("\nFine-Tuned Model ROUGE Scores:")
print(rouge_finetuned)

Original Model ROUGE Scores:
{'rouge1': 0.3205922848345746, 'rouge2': 0.10185477335978205, 'rougeL': 0.18819166141800198, 'rougeLsum': 0.252078246102294}

Fine-Tuned Model ROUGE Scores:
{'rouge1': 0.432548540590151, 'rouge2': 0.16073562916243178, 'rougeL': 0.24437127658137536, 'rougeLsum': 0.3709066102489379}
